## Tugas: Implementasi Naive Bayes Kategorial untuk Prediksi Cuaca Besok  
### Nama: Rahmat Mulia  
### NIM: 235520110141  
### Unit: A (2023)

### Repository GitHub  
Kode dan notebook dapat diakses di:  
https://github.com/skimatt/Naive_Bayes

### Deskripsi Singkat  
Pada tugas ini dilakukan implementasi algoritma Naive Bayes untuk memprediksi apakah besok akan hujan berdasarkan data cuaca kategorial. Proses dilakukan secara manual menggunakan Python, dimulai dari perhitungan *prior* untuk setiap kelas (Yes/No), menghitung *likelihood* menggunakan Laplace smoothing, menghitung skor kelas, menentukan probabilitas *posterior*, hingga menghasilkan label prediksi untuk beberapa contoh kasus cuaca. Seluruh langkah dikerjakan tanpa menggunakan library *sklearn.naive_bayes*, sesuai arahan tugas.


In [41]:
# Import library utama 
import pandas as pd
import numpy as np

In [42]:
# Membaca dataset cuaca kategorial
df = pd.read_excel("dataset_hujan_besok_kategori.xlsx")

# Menampilkan 5 baris pertama
display(df.head())

# Menampilkan informasi struktur data
df.info()

,Temp,Humidity,Wind,Cloud,Pressure,RainToday,RainTomorrow
0,panas,lembab,tenang,cerah,rendah,No,No
1,panas,lembab,pelan,mendung,rendah,Yes,Yes
2,panas,kering,kencang,cerah,tinggi,No,No
3,dingin,lembab,tenang,mendung,rendah,No,No
4,dingin,kering,pelan,cerah,tinggi,Yes,No


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Temp          20 non-null     object
 1   Humidity      20 non-null     object
 2   Wind          20 non-null     object
 3   Cloud         20 non-null     object
 4   Pressure      20 non-null     object
 5   RainToday     20 non-null     object
 6   RainTomorrow  20 non-null     object
dtypes: object(7)
memory usage: 1.2+ KB


### Penjelasan Dataset

Dataset *dataset_hujan_besok_kategori.xlsx* berisi data kondisi cuaca harian dengan fitur-fitur kategorial.  
Berikut penjelasan setiap kolom:

- **Temp** — kondisi temperatur: *dingin*, *sedang*, atau *panas*.  
- **Humidity** — tingkat kelembapan: *kering*, *normal*, atau *lembab*.  
- **Wind** — kondisi angin: *tenang*, *pelan*, atau *kencang*.  
- **Cloud** — kondisi awan: *cerah*, *berawan*, atau *mendung*.  
- **Pressure** — tekanan udara: *rendah*, *normal*, atau *tinggi*.  
- **RainToday** — apakah hari ini hujan: *Yes* atau *No*.  
- **RainTomorrow** — target prediksi, yaitu apakah besok akan hujan (*Yes* atau *No*).

Target atau variabel kelas dalam tugas ini adalah **RainTomorrow**.


In [43]:
# Menghitung jumlah data untuk setiap kelas
yes_count = len(df[df["RainTomorrow"] == "Yes"])
no_count  = len(df[df["RainTomorrow"] == "No"])

# Menghitung total data
N = len(df)

# Menghitung prior
P_yes = yes_count / N
P_no  = no_count / N

# Menampilkan hasil
print("Jumlah data:", N)
print("Jumlah Yes :", yes_count)
print("Jumlah No  :", no_count)
print("\nPrior Probabilities:")
print("P(Yes) =", P_yes)
print("P(No)  =", P_no)

Jumlah data: 20
Jumlah Yes : 8
Jumlah No  : 12

Prior Probabilities:
P(Yes) = 0.4
P(No)  = 0.6


### Penjelasan Perhitungan Prior

Perhitungan prior dilakukan untuk mengetahui peluang awal terjadinya hujan besok tanpa mempertimbangkan fitur apa pun.  
Hasil perhitungan menunjukkan:

- **P(Yes)** adalah peluang besok akan hujan.  
- **P(No)** adalah peluang besok tidak hujan.

Nilai prior ini akan dikalikan dengan *likelihood* pada tahap berikutnya untuk mendapatkan skor dan probabilitas *posterior*.


In [44]:
# Fungsi untuk menghitung daftar kategori unik pada satu fitur
def hitung_kategori_unik(df, nama_kolom):
    return df[nama_kolom].unique()

### Fungsi hitung_kategori_unik
Fungsi ini digunakan untuk memperoleh daftar kategori unik dalam satu fitur (kolom).  
Nilai ini diperlukan untuk menghitung **K**, yaitu jumlah kategori pada fitur tersebut, dalam rumus Laplace smoothing.


In [45]:
# Fungsi untuk menghitung likelihood dengan Laplace smoothing
def hitung_p_val_kelas(df, feature, nilai, kelas, col_target="RainTomorrow"):
    K = len(hitung_kategori_unik(df, feature))  # jumlah kategori unik
    
    numerator = len(df[(df[feature] == nilai) & (df[col_target] == kelas)])
    denominator = len(df[df[col_target] == kelas])
    
    # Laplace smoothing
    return (numerator + 1) / (denominator + K)

### Fungsi hitung_p_val_kelas
Fungsi ini menghitung nilai likelihood **P(feature = nilai | kelas)** dengan menggunakan Laplace smoothing.

Rumus:
\[
P(val | kelas) = \frac{count + 1}{count\_kelas + K}
\]

Penjelasan variabel:
- **count** → jumlah baris dengan feature = nilai dan RainTomorrow = kelas  
- **count_kelas** → banyak data pada kelas tertentu (Yes/No)  
- **K** → jumlah kategori unik pada kolom feature  

Laplace smoothing mencegah probabilitas bernilai 0 sehingga model tetap stabil.


In [46]:
print("P(Temp='sedang' | No )      :", hitung_p_val_kelas(df, "Temp", "sedang", "No"))
print("P(Humidity='lembab' | Yes ) :", hitung_p_val_kelas(df, "Humidity", "lembab", "Yes"))

P(Temp='sedang' | No )      : 0.2
P(Humidity='lembab' | Yes ) : 0.9


### Pengujian Fungsi Likelihood
Bagian ini menguji fungsi Laplace smoothing dengan dua contoh perhitungan, seperti yang diminta pada instruksi PDF.

Contoh diuji:
- P(Temp = “sedang” | RainTomorrow = No)
- P(Humidity = “lembab” | RainTomorrow = Yes)

Hasilnya dapat digunakan untuk membandingkan konsep perhitungan manual di kelas.


In [47]:
# Fungsi utama untuk menghitung skor, posterior, dan prediksi
def predict_naive_bayes(x):
    classes = ["Yes", "No"]
    
    # Menyimpan skor setiap kelas
    scores = {}

    for c in classes:
        product_c = 1
        
        # Untuk setiap fitur dalam dictionary input
        for feature in x:
            nilai = x[feature]
            p = hitung_p_val_kelas(df, feature, nilai, c)  # likelihood dengan smoothing
            product_c *= p
        
        # Skor kelas = prior * produk likelihood
        prior_c = P_yes if c == "Yes" else P_no
        scores[c] = prior_c * product_c
    
    # Hitung posterior
    score_yes = scores["Yes"]
    score_no  = scores["No"]
    P_yes_post = score_yes / (score_yes + score_no)
    P_no_post  = score_no  / (score_yes + score_no)

    # Tentukan prediksi
    pred = "Yes" if P_yes_post > P_no_post else "No"
    
    return pred, P_yes_post, P_no_post

### Fungsi predict_naive_bayes

Fungsi ini melakukan perhitungan Naive Bayes lengkap sesuai instruksi PDF, yaitu:

1. **Menghitung likelihood setiap fitur** menggunakan fungsi Laplace smoothing.
2. **Mengalikan seluruh likelihood** untuk mendapatkan *product likelihood* per kelas.
3. **Menghitung skor** dengan rumus:
   \[
   \text{score} = \text{prior} \times \prod likelihood
   \]
4. **Menghitung posterior**:
   \[
   P\_{yes} = \frac{score\_{yes}}{score\_{yes} + score\_{no}}
   \]
5. **Menentukan prediksi**, yaitu kelas dengan probabilitas posterior terbesar.

Fungsi ini mengembalikan:
- label prediksi ("Yes" atau "No")  
- nilai P\_yes  
- nilai P\_no  

Dengan demikian seluruh alur *Prior → Likelihood → Skor → Posterior → Prediksi* sudah terpenuhi.


In [48]:
# Kasus Uji 1 (contoh sama seperti yang digunakan di kelas)
case1 = {
    "Temp": "sedang",
    "Humidity": "lembab",
    "Wind": "tenang",
    "Cloud": "mendung",
    "Pressure": "rendah",
    "RainToday": "Yes"
}

# Kasus Uji 2 (bebas)
case2 = {
    "Temp": "panas",
    "Humidity": "kering",
    "Wind": "pelan",
    "Cloud": "cerah",
    "Pressure": "tinggi",
    "RainToday": "No"
}

# Kasus Uji 3 (bebas)
case3 = {
    "Temp": "dingin",
    "Humidity": "normal",
    "Wind": "kencang",
    "Cloud": "berawan",
    "Pressure": "normal",
    "RainToday": "Yes"
}

# Menjalankan prediksi dengan interpretasi kalimat
kasus_list = [case1, case2, case3]

for i, case in enumerate(kasus_list, start=1):
    pred, p_yes, p_no = predict_naive_bayes(case)

    print(f"\n=== Kasus Uji {i} ===")
    print("Kondisi Cuaca:", case)
    print(f"P(Yes) = {p_yes:.4f}")
    print(f"P(No)  = {p_no:.4f}")

    # Interpretasi natural sesuai contoh PDF
    kondisi_str = ", ".join([f"{k}={v}" for k, v in case.items()])
    if pred == "Yes":
        print(
            f"Untuk kondisi {kondisi_str}, model memprediksi RainTomorrow=Yes "
            f"dengan probabilitas {p_yes:.4f}."
        )
    else:
        print(
            f"Untuk kondisi {kondisi_str}, model memprediksi RainTomorrow=No "
            f"dengan probabilitas {p_no:.4f}."
        )


=== Kasus Uji 1 ===
Kondisi Cuaca: {'Temp': 'sedang', 'Humidity': 'lembab', 'Wind': 'tenang', 'Cloud': 'mendung', 'Pressure': 'rendah', 'RainToday': 'Yes'}
P(Yes) = 0.9851
P(No)  = 0.0149
Untuk kondisi Temp=sedang, Humidity=lembab, Wind=tenang, Cloud=mendung, Pressure=rendah, RainToday=Yes, model memprediksi RainTomorrow=Yes dengan probabilitas 0.9851.

=== Kasus Uji 2 ===
Kondisi Cuaca: {'Temp': 'panas', 'Humidity': 'kering', 'Wind': 'pelan', 'Cloud': 'cerah', 'Pressure': 'tinggi', 'RainToday': 'No'}
P(Yes) = 0.0010
P(No)  = 0.9990
Untuk kondisi Temp=panas, Humidity=kering, Wind=pelan, Cloud=cerah, Pressure=tinggi, RainToday=No, model memprediksi RainTomorrow=No dengan probabilitas 0.9990.

=== Kasus Uji 3 ===
Kondisi Cuaca: {'Temp': 'dingin', 'Humidity': 'normal', 'Wind': 'kencang', 'Cloud': 'berawan', 'Pressure': 'normal', 'RainToday': 'Yes'}
P(Yes) = 0.7928
P(No)  = 0.2072
Untuk kondisi Temp=dingin, Humidity=normal, Wind=kencang, Cloud=berawan, Pressure=normal, RainToday=Yes, mode

### Pengujian Program Naive Bayes (3 Kasus Uji)

Bagian ini menguji model Naive Bayes untuk tiga kondisi cuaca berbeda:

1. **Kasus 1**  
   Kondisi cuaca yang sama seperti contoh yang digunakan pada perhitungan manual di kelas.

2. **Kasus 2 dan 3**  
   Dua contoh tambahan yang ditentukan sendiri(bebas).  

Untuk setiap kasus, program menampilkan:
- kondisi cuacanya,
- nilai probabilitas P(Yes) dan P(No),
- label prediksi,
- interpretasi dalam bentuk kalimat natural

